In [69]:
import cv2
import os
import numpy as np
from PIL import Image
from skimage.metrics import structural_similarity

In [70]:
#reads in video to init_video
with open("dataset/Videos/data_test1.rgb", "rb") as f:
    v = np.fromfile(f, np.dtype('B'))
#setting the number of pixels in frame
pxct = 129600
#setting the number of frames
frct = (int)(v.size/(3*pxct))
init_video = []
for frame_number in range(frct):
        start = (frame_number) * pxct * 3
        curr_frame = v[start:start + 3 * pxct]
        final_frame = np.reshape(curr_frame, (480,270,3), order = 'F')
        final_frame = np.rot90(final_frame,3)
        final_frame = np.flip(final_frame,1)
        
        #searching these final frames for logos (new cell)
        init_video.append(final_frame)

In [71]:
#generate cuts for advertisement
advert_frames = []
for frame_number in range(frct - 1):
    #calculate new shots with histogram
    #less means more similar
    hist1 = cv2.calcHist([init_video[frame_number]],[0],None,[256],[0,256])
    hist2 = cv2.calcHist([init_video[frame_number + 1]],[0],None,[256],[0,256])
    score=cv2.compareHist(hist1,hist2,cv2.HISTCMP_BHATTACHARYYA)
    #store score for ssi
    #less means more similar
    #i1 = cv2.cvtColor(init_video[frame_number], cv2.COLOR_BGR2GRAY)
    #i2 = cv2.cvtColor(init_video[frame_number + 1], cv2.COLOR_BGR2GRAY)
    #score2 = structural_similarity(i1, i2)
    #score2 = 1-score2 
    #frame_stats[frame_number][1] = score2
    
    #score indicates sensitivity for shot changes
    if score > 0.3:
        advert_frames.append(frame_number)
        advert_frames.append(frame_number+1)
        #saves the image, currently used for debugging
        #if score + score2 > 0.3:
        data = Image.fromarray(init_video[frame_number]).convert('RGB')
        data.save('images_cut/{}.png'.format(frame_number))
        data = Image.fromarray(init_video[frame_number+1]).convert('RGB')
        data.save('images_cut/{}.png'.format(frame_number+1))
#save beginning and end of cuts in number of frames
#storing the advertisements to replace with
#indexed the same way as that of cuts_begin and cuts_end
#if not to be replaced, a blank line
replaced_advertisements = []
cuts_begin = []
cuts_end = []
for i in range(len(advert_frames)):
    for j in range(i,len(advert_frames)):
        if advert_frames[j] - advert_frames[i] == 449:
            cuts_begin.append(advert_frames[i])
            cuts_end.append(advert_frames[j])

In [8]:
#image negatives for training
# for f in range(frct - 1):
#     if f > 3000 or f < 1200:
#         data = Image.fromarray(init_video[f]).convert('RGB')
#         data.save('negative_images/{}.png'.format(f))

In [ ]:
#cuts_begin has all starts of clips of 15seconds (449 frames) long exactly
#cuts_end has all the ends
print(advert_frames)
cuts_begin = []
cuts_end = []
for i in range(len(advert_frames)):
    for j in range(i,len(advert_frames)):
        if advert_frames[j] - advert_frames[i] == 449:
            cuts_begin.append(advert_frames[i])
            cuts_end.append(advert_frames[j])
print(cuts_begin)            

In [73]:
#reading in all other advertisements
#saves in a dictionary with first part assumed to be the company name

advertisements = {}
PATHS_TO_ADVERTISEMENTS = "dataset/Ads"
for file in os.listdir(PATHS_TO_ADVERTISEMENTS):
    filename = os.fsdecode(file)
    if filename.endswith(".rgb"): 
        vpath = os.path.join(PATHS_TO_ADVERTISEMENTS, filename)
        with open(vpath, "rb") as f:
            v = np.fromfile(f, np.dtype('B'))
        #setting the number of pixels in frame
        pxct = 129600
        #setting the number of frames
        frct = (int)(v.size/(3*pxct))
        video = []
        for frame_number in range(frct):
            start = (frame_number) * pxct * 3
            curr_frame = v[start:start + 3 * pxct]
            final_frame = np.reshape(curr_frame, ( 480,270, 3), order = 'F')
            final_frame = np.rot90(final_frame,3)
            final_frame = np.flip(final_frame,1)
            video.append(final_frame)
        #Get the name of the company
        key = filename.split('_')[0]
        advertisements[key] = video


    

In [74]:
#splicing in videos with the names 
#save beginning and end of cuts in number of frames

#storing the advertisements to replace with
#indexed the same way as that of cuts_begin and cuts_end
#if not to be replaced, an empty string
replaced_advertisements = ["Subway", "Starbucks"]
final_video = []
for i in range (len(cuts_begin)):
    if i == 0:
        final_video = init_video[:cuts_begin[i]] + advertisements[replaced_advertisements[i]] + init_video[cuts_end[i]:]
    else:
        #further work required to sync frames iff replacement ad lengths are variable
        final_video = final_video[:cuts_begin[i]] + advertisements[replaced_advertisements[i]] + final_video[cuts_end[i]:]
        

In [217]:
#experimenting with SIFT
#finding points and comparing across image 
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
#returns boolean if logo of logo_path is recognized in frame
def logo_recognition(frame, logo):
    #read in logo 
    graylogo = cv2.cvtColor(logo, cv2.COLOR_BGR2GRAY)
#cv2.imshow('image', graylogo)
#cv2.waitKey(5000)

# #read in frame from video
    #frame = Image.fromarray(final_video[2100]).convert('RGB')

# #reformat
    #frame.save('images/box-test.png'.format(1));
    #frame1 = cv2.imread('images/box-test.png')

# #change to grayscale
    grayframe = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    sift = cv2.ORB_create(nfeatures=2000, patchSize=25)
    logopoints, logodescriptors = sift.detectAndCompute(graylogo, None)
    framepoints, framedescriptors = sift.detectAndCompute(grayframe, None)

#     bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    bf = cv2.BFMatcher(cv2.NORM_HAMMING)
#     bf = cv2.BFMatcher()
#     print(bf)
#     matches = bf.match(logodescriptors,framedescriptors)
    matches = bf.knnMatch(logodescriptors,framedescriptors, k=2)

#     matches = sorted(matches, key = lambda x:x.distance)

    # Apply ratio test
    good_points = []
    for m,n in matches:
       if m.distance < 0.7*n.distance:
           good_points.append([m])
    print(good_points)

# cv2.drawMatchesKnn expects list of lists as matches.
    img3 = cv2.drawMatchesKnn(logo, logopoints, frame, framepoints, good_points, None,
                              matchColor=(0, 255, 0), matchesMask=None,
                              singlePointColor=(255, 0, 0), flags=0)

    plt.imshow(img3),plt.show()
    return (len(good_points) > 20)







In [ ]:
#create the video
#file path, format, frames per second, width and height
out = cv2.VideoWriter('video.avi', cv2.VideoWriter_fourcc(*'DIVX'), 30, (480, 270))
#RGBA format
#frames = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60]    
#store all logos recognized in replaced_advertisements[]
for i in range(len(final_video)):
    #cv2.imshow('images/box-1.png', img)
    data = Image.fromarray(final_video[i]).convert('RGB')
    #print(data)
    #convert rgb frame to png for writing
    data.save('images/box-test.png'.format(1));
    #get new saved png frame
    img = cv2.imread('images/box-test.png')
    #call the sift algorithm
    #if there are matches, we return the frames that they exist in
    logo_names = ["subway", "starbucks"]
    for logo in logo_names:
        if(logo_recognition(img,logo)):
            replaced_advertisements.append(logo)

    #write on image here
#    if(i in frames):
# #draw rectangle on frame
#         cv2.rectangle(img, (10,10), (100, 100), (0, 255, 0), 2)
# #just to verify box comes out in right spot
#         cv2.imwrite("images/test-box.png", img)

#     #change back to rgb for avi video
#         #img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#         out.write(img)
#     else:
    #out.write(img)

#out.release()

In [ ]:
logo_names = ["subway", "starbucks"]
logos = []
replaced_advertisements = []
for l in logo_names:
    logos.append(plt.imread('Dataset/Brand Images/{company}_logo.bmp'.format(company = l)))
for k in range(len(cuts_begin)):
    loopstart = 0
    if not k == 0:
        loopstart = cuts_end[k-1]
    for j in range(loopstart,cuts_begin[k],5):
        img = np.flip(final_video[j],axis = 2)
        #cv2.imshow('images/box-1.png', img)
        #data = Image.fromarray(final_video[i]).convert('RGB')
        #print(data)
        #convert rgb frame to png for writing
        #data.save('images/box-test.png'.format(1));
        #get new saved png frame
        #img = cv2.imread('images/box-test.png')
        #call the sift algorithm
        #if there are matches, we return the frames that they exist in
        tobreak = False
        for logo in range(len(logos)):
            if(logo_recognition(img,logos[logo])):
                replaced_advertisements.append(logo_names[logo])
                print(logo_names[logo])
                tobreak = True
                break
        if tobreak:
            break
                

In [ ]:
f = open("cuts_begin.txt", "w")
f.write(cuts_begin)
f.close()

In [ ]:
replaced_advertisements


In [78]:
#saves data for java to cut audio
ad_times = open("ad_times.txt", "w")
for i in range(len(replaced_advertisements)):
    ad_times.write(replaced_advertisements[i] + " " + str(cuts_begin[i]) + "\n")
ad_times.close()

In [80]:
#add function to find points and draw the box for each frame in the video
#output video to rgb
#
with open('output.rgb', "wb") as f:
    for p in final_video:
        output_frame = np.ravel(np.split(p, 3, axis = 2), order = 'C')
        f.write (bytes(output_frame))
f.close()

In [14]:
logo_cascade = cv2.CascadeClassifier('data/cascade.xml')
img = cv2.imread('class/2040.png')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
logos = logo_cascade.detectMultiScale(gray,1.3, 5)
for (x,y,w,h) in logos:
    cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    cv2.imshow('img',img)
#data.save('images/{}.png'.format(1337))


In [63]:
#output avi for debugging
out = cv2.VideoWriter('video.avi', cv2.VideoWriter_fourcc(*'DIVX'), 30, (480, 270))
for i in range(len(final_video)):
    out.write(final_video[i])
out.release()